In [ ]:
# Make sure you have the latest version of the SDK available to use the Batch API
# %pip install openai --upgrade

In [2]:
import json
from openai import OpenAI
import pandas as pd
from IPython.display import Image, display
from dotenv import load_dotenv
import os

In [4]:
# load openai API key from .env
load_dotenv()

True

In [5]:
# Initializing OpenAI client - see https://platform.openai.com/docs/quickstart?context=python
client = OpenAI()

# Prepare the data for the batch API

In [91]:
# 合併所有的cleaned_text和video_title，準備作為批次上傳的data
"""
要分成三批
1. v0-v15 跳過10, 11, 12, 16, 17
2. v18-v35 跳過v30, 31, 32
3. v30-31
"""

all_comments = []

for i in range(18, 36):
    # if i in [10, 11, 12, 16, 17]:
    #     print(f"跳過 video_{i}")
    #     continue

    if i in [30, 31, 32]:
        print(f"跳過 video_{i}")
        continue
    
    file_path = f"for_gpt_tag/video_{i}_filtered_spam.csv"
    # file_path = f"spam_tag/comments_spam_tag.csv"
    df = pd.read_csv(file_path, encoding='utf-8-sig')

    for index, row in df.iterrows():
        # print(f"Processing row {index + 1} of {len(df)}")
        video_title = row['video_title']
        cleaned_text = row['cleaned_text']
        all_comments.append({
            "video_title": video_title,
            "comment": cleaned_text
        })

# Save the combined data to a JSON file
with open('for_gpt_tag/ver2_comments_for_tag.json', 'w', encoding='utf-8') as f:
    json.dump(all_comments, f, ensure_ascii=False, indent=4)

跳過 video_30
跳過 video_31
跳過 video_32


In [92]:
# check length of all_comments
print(f"Total comments processed: {len(all_comments)}")

Total comments processed: 44249


# 從instruction2之後只要讀檔

1. for_gpt_tag/ver1_comments_for_tag.json
2. for_gpt_tag/ver2_comments_for_tag.json
3. for_gpt_tag/ver3_comments_for_tag.json

In [33]:
i = 3
with open(f'for_gpt_tag/ver{i}_comments_for_tag.json', 'r', encoding='utf-8') as f:
    all_comments = json.load(f)

print(len(all_comments))

46696


In [34]:
# set up batch input
"""
分別上傳instruction1, instruction2, instruction3的batch_input

custom_id: instruction"1"-task-0, instruction"2"-task-1, instruction"3"-task-2
messages:[
{
    "role": "system", "content": instruction"1"
},
{
    "role": "user", "content": f"【影片標題】{item['video_title']}\n【留言內容】{item['comment']}"
}
]
    
"""
from gpt_instructions import *

tasks = []

for index, item in enumerate(all_comments):
    task = {
        "custom_id": f"instruction3-task-{index+86284}", # ver3 : 86284
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4.1-mini",
            "response_format": { 
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system", "content": instruction3
                },
                {
                    "role": "user", "content":
                        f"【影片標題】{item['video_title']}\n【留言內容】{item['comment']}"
                }
            ],
            "temperature": 0
        }
    }
    tasks.append(task)

In [35]:
# Creating the file
file_name = "batch_input/instruction3_ver3_comments.jsonl"

with open(file_name, 'w', encoding='utf-8') as file:
    for obj in tasks:
        file.write(json.dumps(obj, ensure_ascii=False) + '\n')

# 檢查當前所有Batch

In [7]:
# 檢查所有的batch jobs
client.batches.list()

SyncCursorPage[Batch](data=[Batch(id='batch_684bf6bbee048190819505b124197ea4', completion_window='24h', created_at=1749808827, endpoint='/v1/chat/completions', input_file_id='file-3nxJghe61tFku5zu4uYiNm', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1749895227, failed_at=None, finalizing_at=None, in_progress_at=1749808849, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=44249)), Batch(id='batch_684bf5a6e3c88190bb3852e848859c35', completion_window='24h', created_at=1749808550, endpoint='/v1/chat/completions', input_file_id='file-4FC84SMm2N6bXJXxDZuwYv', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1749894950, failed_at=None, finalizing_at=None, in_progress_at=1749808562, metadata=None, output_file_id=None, request_co

# Upload batch input (jsonl)

In [36]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [37]:
print(batch_file)

FileObject(id='file-DSJ2HH98pjGk8iKGPaTLZW', bytes=79469205, created_at=1749814865, filename='instruction3_ver3_comments.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)


# Create Batch

In [38]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

# Check Batch Status

In [39]:
batch = client.batches.retrieve(batch_job.id)
print(batch)

Batch(id='batch_684c0e972dec819097916ce046138e9f', completion_window='24h', created_at=1749814935, endpoint='/v1/chat/completions', input_file_id='file-DSJ2HH98pjGk8iKGPaTLZW', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1749901335, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


### Check Batch Progress

In [53]:
batch_id_dict = {
    "instruction1_v1" : "batch_684be834800481909e086701c7a3ebab",
    "instruction1_v2" : "batch_684bed4d5f70819095f2e5da49f2dd25",
    "instruction1_v3" : "batch_684bf015d7388190823e100c889fb8e8",
    "instruction2_v1" : "batch_684bf5a6e3c88190bb3852e848859c35",
    "instruction2_v2" : "batch_684bf6bbee048190819505b124197ea4",
    "instruction2_v3" : "batch_684bf76e1a888190b5785faaffda5a0a",
    "instruction3_v1" : "batch_684c0bb660f48190a244b302bc4f82c1",
    "instruction3_v2" : "batch_684c0d89795c8190a8e98210f6ba20a8",
    "instruction3_v3" : "batch_684c0e972dec819097916ce046138e9f"
}

In [99]:
import time

def check_batch_status(batch_id):
    """檢查批次處理狀態"""
    batch = client.batches.retrieve(batch_id)
    print(f"批次狀態: {batch.status}")
    print(f"已完成: {batch.request_counts.completed}")
    print(f"失敗: {batch.request_counts.failed}")
    print(f"總計: {batch.request_counts.total}")
    return batch

# 定期檢查狀態
batch_id = batch_id_dict["instruction3_v3"]

# batch_id = batch_job.id
print(f"批次ID: {batch_id}")

current_batch = check_batch_status(batch_id)

批次ID: batch_684c0e972dec819097916ce046138e9f
批次狀態: completed
已完成: 46696
失敗: 0
總計: 46696


# Cancel Batch

In [8]:
client.batches.cancel("batch_684bf30244cc8190bfce0d1a6783a3ee")

Batch(id='batch_684bf30244cc8190bfce0d1a6783a3ee', completion_window='24h', created_at=1749807874, endpoint='/v1/chat/completions', input_file_id='file-NseY25fx744Upm9RzYxNBt', object='batch', status='cancelling', cancelled_at=None, cancelling_at=1749808410, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1749894274, failed_at=None, finalizing_at=None, in_progress_at=1749807884, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=1401, failed=0, total=46696))

# View ourput result

In [ ]:
# 直接使用 DataFrame 收集資料
import re
import pandas as pd
import os

# 確保輸出資料夾存在
os.makedirs('batch_outputs', exist_ok=True)

# 創建空的 DataFrame
df_results = pd.DataFrame(columns=['task_id', 'video_title', 'comment'])

for vi in range(1, 4):
    input_path = f"batch_input/instruction1_ver{vi}_comments.jsonl"
    
    if os.path.exists(input_path):
        print(f">>> 正在處理檔案: instruction1_ver{vi}_comments.jsonl")
        
        batch_data = []  # 用來收集當前檔案的資料
        
        with open(input_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    try:
                        data = json.loads(line)
                        
                        # 1. 提取 task_id (從 custom_id 中提取)
                        custom_id = data['custom_id']
                        task_id = custom_id.split('-', 1)[1]
                        
                        # 2. 提取 content
                        content = data['body']['messages'][1]['content']
                        
                        # 3. 使用正則表達式提取 video_title 和 comment
                        video_title_match = re.search(r'【影片標題】(.*?)【留言內容】', content, re.DOTALL)
                        video_title = video_title_match.group(1).strip() if video_title_match else ""
                        
                        comment_match = re.search(r'【留言內容】(.*)', content, re.DOTALL)
                        comment = comment_match.group(1).strip() if comment_match else ""
                        
                        # 加入批次資料
                        batch_data.append({
                            'task_id': task_id,
                            'video_title': video_title,
                            'comment': comment
                        })
                        
                    except json.JSONDecodeError as e:
                        print(f"JSON 解析錯誤在檔案 {input_path}: {e}")
                    except Exception as e:
                        print(f"處理錯誤在檔案 {input_path}: {e}")
        
        # 將批次資料加入主 DataFrame
        if batch_data:
            batch_df = pd.DataFrame(batch_data)
            df_results = pd.concat([df_results, batch_df], ignore_index=True)
            print(f"    處理了 {len(batch_data)} 筆資料")
    else:
        print(f"檔案不存在: {input_path}")

print(f"\n總共處理了 {len(df_results)} 筆資料")

# 保存到 CSV
df_results.to_csv('batch_outputs/tag_results.csv', index=False, encoding='utf-8-sig')

print(">>> 資料已保存到 batch_outputs/tag_results.csv")

# 顯示前幾筆資料
print("\n=== 前5筆資料預覽 ===")
print(df_results.head())

>>> 正在處理檔案: instruction1_ver1_comments.jsonl
    處理了 42035 筆資料
>>> 正在處理檔案: instruction1_ver2_comments.jsonl
    處理了 44249 筆資料
>>> 正在處理檔案: instruction1_ver3_comments.jsonl
    處理了 46696 筆資料

總共處理了 132980 筆資料
>>> 資料已保存到 batch_outputs/tag_results.csv

=== 前5筆資料預覽 ===
  task_id                      video_title                 comment
0  task-0  【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會                  太有活了贵司
1  task-1  【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會       我的天这段影片的评论区是发生了什么
2  task-2  【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會  原来这就是之后节目里提到的被炎上的事件之一吗
3  task-3  【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會        突然發現低卡的夥伴們ᶘ ᵒᴥᵒᶅ
4  task-4  【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會         怎麼選舉可以有這麼多好笑的內容


In [4]:
df_results = pd.read_csv('batch_outputs/tag_results.csv', encoding='utf-8-sig')

In [5]:
# 更高效的方法：使用字典收集資料，最後一次性合併

import json
import pandas as pd
import os
from collections import defaultdict

# 使用字典來儲存結果，key 是 task_id
results_dict = defaultdict(dict)

print("開始處理批次結果...")

for i in range(1,4):  # instruction1, instruction2, instruction3
    for vi in range(1, 4):  # v1, v2, v3
        # if i == 3 and vi == 3:
        #     continue
            
        output_path = f"batch_outputs/instruction{i}_v{vi}_results.jsonl"
        
        if not os.path.exists(output_path):
            print(f"檔案不存在: {output_path}")
            continue
            
        print(f">>> 處理 instruction{i}_v{vi}")
        
        processed_count = 0
        with open(output_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    try:
                        data = json.loads(line)
                        
                        task_id = data['custom_id'].split('-', 1)[1]
                        
                        # 解析 GPT 回應
                        result = data['response']['body']['choices'][0]['message']['content']
                        result_json = json.loads(result)
                        tag = result_json.get('tag', '')
                        reason = result_json.get('reason', '')
                        
                        # 直接存入字典，避免 DataFrame 查找
                        results_dict[task_id][f'tag{i}'] = tag
                        results_dict[task_id][f'reason{i}'] = reason
                        
                        processed_count += 1
                        
                        # 每 1000 筆顯示進度
                        if processed_count % 1000 == 0:
                            print(f"    已處理 {processed_count} 筆")
                            
                    except (json.JSONDecodeError, KeyError) as e:
                        print(f"處理錯誤: {e}")
        
        print(f">>> 完成，共處理 {processed_count} 筆")

# 將字典資料合併到原始 DataFrame
print("合併資料到 DataFrame...")

# 為 df_results 添加新欄位
for col in ['tag1', 'reason1', 'tag2', 'reason2', 'tag3', 'reason3']:
    if col not in df_results.columns:
        df_results[col] = None

# 一次性更新所有資料
for idx, row in df_results.iterrows():
    task_id = row['task_id']
    if task_id in results_dict:
        for key, value in results_dict[task_id].items():
            df_results.at[idx, key] = value
    
    # 顯示進度
    if (idx + 1) % 5000 == 0:
        print(f"已合併 {idx + 1}/{len(df_results)} 筆資料")

# 保存結果
df_results.to_csv('batch_outputs/tag_results_combined.csv', index=False, encoding='utf-8-sig')
print("✅ 完成！")

開始處理批次結果...
>>> 處理 instruction1_v1
    已處理 1000 筆
    已處理 2000 筆
    已處理 3000 筆
    已處理 4000 筆
    已處理 5000 筆
    已處理 6000 筆
    已處理 7000 筆
    已處理 8000 筆
    已處理 9000 筆
    已處理 10000 筆
    已處理 11000 筆
    已處理 12000 筆
    已處理 13000 筆
    已處理 14000 筆
    已處理 15000 筆
    已處理 16000 筆
    已處理 17000 筆
    已處理 18000 筆
    已處理 19000 筆
    已處理 20000 筆
    已處理 21000 筆
    已處理 22000 筆
    已處理 23000 筆
    已處理 24000 筆
    已處理 25000 筆
    已處理 26000 筆
    已處理 27000 筆
    已處理 28000 筆
    已處理 29000 筆
    已處理 30000 筆
    已處理 31000 筆
    已處理 32000 筆
    已處理 33000 筆
    已處理 34000 筆
    已處理 35000 筆
    已處理 36000 筆
    已處理 37000 筆
    已處理 38000 筆
    已處理 39000 筆
    已處理 40000 筆
    已處理 41000 筆
    已處理 42000 筆
>>> 完成，共處理 42035 筆
>>> 處理 instruction1_v2
    已處理 1000 筆
    已處理 2000 筆
    已處理 3000 筆
    已處理 4000 筆
    已處理 5000 筆
    已處理 6000 筆
    已處理 7000 筆
    已處理 8000 筆
    已處理 9000 筆
    已處理 10000 筆
    已處理 11000 筆
    已處理 12000 筆
    已處理 13000 筆
    已處理 14000 筆
    已處理 15000 筆
    已處理 16000 筆
    已處理 17000

In [24]:
len(df_results)

125979

In [6]:
# 過濾掉len(comments) >128
df_filtered = df_results[df_results['comment'].str.len() <= 128]
print(f">>> 過濾後剩下：{len(df_filtered)}")

df_filtered.to_csv('batch_outputs/tag_results_filtered.csv', index=False, encoding='utf-8-sig')

>>> 過濾後剩下：125979


# 檢查每一部影片中的廢話

- if tag1, tag2, ta3 有兩個都是 true ：['spam_tag'] == "spam"
- else： ['spam_tag'] == "non_spam"

In [7]:
df = pd.read_csv('batch_outputs/tag_results_filtered.csv', encoding='utf-8-sig')

def spam_tag(row):
    """判斷是否為垃圾留言"""
    spam_count = 0
    if row['tag1'] == True:
        spam_count += 1
    if row['tag2'] == True:
        spam_count += 1
    if row['tag3'] == True:
        spam_count += 1

    if spam_count >= 2:
        df.at[row.name, 'spam_tag'] = 'spam'
    else:
        df.at[row.name, 'spam_tag'] = 'non-spam'
    return df.at[row.name, 'spam_tag']

for index, row in df.iterrows():
    spam_tag(row)
# Save the final DataFrame with spam tags
df.to_csv('batch_outputs/tag_results_final.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 描述性統計：spam/non-spam留言的數量和比例

spam_df = pd.read_csv('batch_outputs/tag_results_final.csv', encoding='utf-8-sig')

usefull_comment_df = spam_df[spam_df['spam_tag'] == 'non-spam']
usefull_comment_df.to_csv('batch_outputs/useful_comments.csv', index=False, encoding='utf-8-sig')

# count spam and non-spam comments by video title
video_spam_stats = spam_df.groupby(['video_title', 'spam_tag']).size().unstack(fill_value=0)
video_spam_stats.columns = ['non_spam_count', 'spam_count']

video_spam_stats['total_comments'] = video_spam_stats['non_spam_count'] + video_spam_stats['spam_count']
video_spam_stats['spam_percentage'] = (video_spam_stats['spam_count'] / video_spam_stats['total_comments'] * 100).round(2)

video_spam_stats.reset_index(inplace=True)
video_spam_stats_sorted = video_spam_stats.sort_values('spam_percentage', ascending=False)

video_spam_stats_sorted.to_csv('batch_outputs/video_spam_statistics.csv', index=False, encoding='utf-8-sig')


## 人工標註1000則 -> 用傳統機器學習的方法評估準確性

In [ ]:
# spam跟non-spam各抽500則做人工抽查

spam_df = df[df['spam_tag'] == 'spam'].sample(n=500, random_state=42)
non_spam_df = df[df['spam_tag'] == 'non-spam'].sample(n=500, random_state=42)
final_sample_df = pd.concat([spam_df, non_spam_df], ignore_index=True)

# 隨機打亂最終的抽樣結果
final_sample_df = final_sample_df.sample(frac=1, random_state=42).reset_index(drop=True)

final_sample_df.to_csv('batch_outputs/result_for_check.csv', index=False, encoding='utf-8-sig')

In [1]:
import pandas as pd
# human tag data
human_tag_df = pd.read_csv("batch_outputs/human_tag_result.csv", encoding='utf-8-sig')

# 合併人工標註和LLM標註結果
gpt_tag_df = pd.read_csv("batch_outputs/tag_results_final.csv", encoding='utf-8-sig')

# 設定 task_id 為 index
gpt_tag_df.set_index('task_id', inplace=True)
human_tag_df.set_index('task_id', inplace=True)

# 只取 human_tag 欄位加入到 gpt_tag_df
merged_df = gpt_tag_df.copy()
merged_df['human_tag'] = human_tag_df['human_tag']

# 如果需要重置 index 回到一般欄位
merged_df.reset_index(inplace=True)

In [2]:
# merged_df.head(5)
merged_df.to_csv("batch_outputs/merged_tag_results.csv", index=False, encoding='utf-8-sig')

In [3]:
# 計算標註一致性

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 範例資料
true_labels = human_tag_df['human_tag']
pred_labels = human_tag_df['spam_tag']

# 計算四個指標（macro平均適用於不平衡資料）
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')
accuracy = accuracy_score(true_labels, pred_labels)

print('Precision（精確率）:', precision)
print('Recall（召回率）:', recall)
print('F1 Score:', f1)
print('Accuracy（準確率）:', accuracy)

# 儲存結果
results = {
    "precision": precision,
    "recall": recall,
    "f1_score": f1,
    "accuracy": accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv("batch_outputs/llm_tagging_metrics.csv", index=False, encoding='utf-8-sig')

Precision（精確率）: 0.845
Recall（召回率）: 0.8672228585053072
F1 Score: 0.8426189825875673
Accuracy（準確率）: 0.845
